In [ ]:
! pip install datasets evaluate transformers transformers[torch] rouge-score nltk --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
import transformers
from datasets import load_dataset, concatenate_datasets
from evaluate import load
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
from huggingface_hub import notebook_login
import numpy as np
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
torch.cuda.set_device(device)
torch.cuda.current_device()

0

In [ ]:
dataset_files = {"tr_anli": "tr_anli.parquet",
                 "tr_fever": "tr_fever.parquet",
                 "tr_ling": "tr_ling.parquet",
                 "tr_mnli": "tr_mnli.parquet",
                 "tr_wanli": "tr_wanli.parquet"}
dict_tr = load_dataset("parquet", data_files=dataset_files)
datasets_tr = [dict_tr['tr_anli'], dict_tr['tr_fever'], dict_tr['tr_ling'], dict_tr['tr_mnli'], dict_tr['tr_wanli']]

Generating tr_anli split: 0 examples [00:00, ? examples/s]

Generating tr_fever split: 0 examples [00:00, ? examples/s]

Generating tr_ling split: 0 examples [00:00, ? examples/s]

Generating tr_mnli split: 0 examples [00:00, ? examples/s]

Generating tr_wanli split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = concatenate_datasets(datasets_tr)
dataset = dataset.remove_columns(["premise_original", "hypothesis_original"])

In [ ]:
dataset

Dataset({
    features: ['label', 'premise', 'hypothesis'],
    num_rows: 105000
})

In [ ]:
label_col = ['gereklilik' if x[1] == 0 else 'nötr' if x[1] == 1 else 'çelişki' for x in enumerate(dataset['label'])]

In [ ]:
dataset = dataset.rename_column("label", "labelnum")
dataset = dataset.add_column("labelstr", label_col)

In [ ]:
dataset_shuffled = dataset.shuffle(seed=42)
dataset_eval = dataset_shuffled.select(range(5000))

In [ ]:
dataset_splitted = dataset_eval

In [ ]:
dataset_splitted[0]

{'labelnum': 0,
 'premise': 'Bir eklektus habitatı nasıl kurulurbr>Mümkün olan en büyük kuş kafesini satın alın. Kullandığınız kafes en az 3 feet genişliğinde (. 91 m), 2 feet derinliğinde (. 6 m) ve 4 feet yüksekliğinde (1.22 m) olmalıdır. Büyük bir kafes sağlamak, kuşunuzun gelişmesi gereken odaya sahip olmasını sağlayacaktır.',
 'hypothesis': '1.22 metre kabaca dört metredir.',
 'labelstr': 'gereklilik'}

In [ ]:
def prompt_format(row):
  return f"hipotez: {row[1]} önerme: {row[0]}"

In [ ]:
max_input_length = 512
max_target_length = 1

def preprocess_function(examples, tokenizer):
  inputs = [prompt_format(x) for x in zip(examples['premise'], examples['hypothesis'])]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
  labels = tokenizer(text_target=examples["labelstr"], max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
metric = load("rouge")

In [ ]:
def compute_metrics_1(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer_1.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer_1.pad_token_id)
    decoded_labels = tokenizer_1.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer_1.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def compute_metrics_2(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer_2.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer_2.pad_token_id)
    decoded_labels = tokenizer_2.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer_2.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
tokenizer_1 = AutoTokenizer.from_pretrained("boun-tabi-LMG/turna_nli_nli_tr")
tokenizer_2 = AutoTokenizer.from_pretrained("aniltepe/turna-nlitr-finetuned-20k-5e")

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model_1 = AutoModelForSeq2SeqLM.from_pretrained("boun-tabi-LMG/turna_nli_nli_tr")
model_2 = AutoModelForSeq2SeqLM.from_pretrained("aniltepe/turna-nlitr-finetuned-20k-5e")

config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
ds_encoded_1 = dataset_splitted.map(lambda x: preprocess_function(x, tokenizer_1), batched=True)
ds_encoded_2 = dataset_splitted.map(lambda x: preprocess_function(x, tokenizer_2), batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
args_1 = Seq2SeqTrainingArguments(
    "trainer_1",
    predict_with_generate=True,
)
trainer_1 = Seq2SeqTrainer(
    model_1,
    args_1,
    eval_dataset=ds_encoded_1,
    tokenizer=tokenizer_1,
    compute_metrics=compute_metrics_1
)

In [ ]:
args_2 = Seq2SeqTrainingArguments(
    "trainer_2",
    predict_with_generate=True,
)
trainer_2 = Seq2SeqTrainer(
    model_2,
    args_2,
    eval_dataset=ds_encoded_2,
    tokenizer=tokenizer_2,
    compute_metrics=compute_metrics_2
)

In [ ]:
trainer_1.evaluate()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.9139402508735657,
 'eval_rouge1': 36.24,
 'eval_rouge2': 36.18,
 'eval_rougeL': 36.23,
 'eval_rougeLsum': 36.22,
 'eval_gen_len': 2.4324,
 'eval_runtime': 2194.5934,
 'eval_samples_per_second': 2.278,
 'eval_steps_per_second': 0.285}

In [ ]:
trainer_2.evaluate()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.7527487277984619,
 'eval_rouge1': 36.76,
 'eval_rouge2': 36.7,
 'eval_rougeL': 36.76,
 'eval_rougeLsum': 36.75,
 'eval_gen_len': 2.4534,
 'eval_runtime': 2196.8721,
 'eval_samples_per_second': 2.276,
 'eval_steps_per_second': 0.284}